In [3]:
import pandas as pd
# import requests
from bs4 import BeautifulSoup
import re

In [4]:
# URL = "data/Micro0 Gonzalo septiembre2021.html"
# page = requests.get(URL)
# print(page.text)

# 1. Get data from .html

In [5]:
# The storage structure is as follows (JSON compatibl-ish):
# 
# {micro_name: [
#       {"workout_desc": "Pierna",
#        "date_workout": session1_date,
#        "exercises": [
#               {"nombre": "sentadilla",
#                "series": 4, 
#                ...},
#               {"nombre": "peso muerto",
#                "series": 4}, 
#               ...
#               ]
#       },
#       {"workout_desc": "Torso",
#        "date_workout": session2_date,
#        "exercises": [
#               {"nombre": "dominadas",
#                "series": 4, 
#                ...},
#               {"nombre": "press banca",
#                "series": 4}, 
#               ...
#               ]
#       },
#       ...
# ] }

with open("data/Micro0 Gonzalo septiembre2021.html") as file:
    soup = BeautifulSoup(file, "html.parser")

    sessions_list = []
    sessions = soup.find_all("div", class_="dia")
    for session in sessions:
        session_dict = {}

        workout_desc = session.find("div", class_="titulo").text.lower()
        session_dict["workout_desc"] = workout_desc

        day = int(session.find("div", id="dia").text)
        month = int(session.find("div", id="mes").text)
        year = int(session.find("div", id="anyo").text)
        date = pd.Timestamp(day=day, month=month, year=year).date()
        session_dict["date_workout"] = date
        
        exercises_list = []
        exercises = session.find("div", class_="cuerpo-boxdia").find_all("div", re.compile("ejercicio.*"))
        for exercise in exercises:
            exercise_dict = {}
            for element in exercise.find_all("div"):
                exercise_dict[element.attrs['class'][0]] = element.text
            exercises_list.append(exercise_dict)
        session_dict["exercises"] = exercises_list

        sessions_list.append(session_dict)

    # exercises_list = []
    # exercises = soup.find(id="1885").find(class_="cuerpo-boxdia").find_all(class_=re.compile("ejercicio.*"))

    # for exercise in exercises:
    #     exercise_dict = {}
    #     for element in exercise.find_all("div"):
    #         exercise_dict[element.attrs['class'][0]] = element.text
    #     exercises_list.append(exercise_dict)
    # print(exercises_list)

    block_name = soup.find(id="microciclo").text
    micro_dict = {block_name: sessions_list}
    # print(micro_dict)
    
    # exercise = soup.find(id="1885").find(class_="cuerpo-boxdia").find(class_="ejercicio10759").find_all("div")
    # for element in exercise:
    #     exercise_dict[element.attrs['class'][0]] = element.text
    # print(exercise_dict)    

In [6]:
next(iter( micro_dict.items() ))
# micro_dict

('Micro 0 Gonzalo septiembre2021',
 [{'workout_desc': 'pierna ',
   'date_workout': datetime.date(2021, 9, 6),
   'exercises': [{'nombre': 'Sentadilla',
     'series': '4',
     'cargas': '0 %',
     'kilos': '70 Kg',
     'repeticiones': '10',
     'rpe': '8',
     'descanso': '2 min'},
    {'nombre': 'Peso muerto',
     'series': '4',
     'cargas': '0 %',
     'kilos': '100 Kg',
     'repeticiones': '5',
     'rpe': '5',
     'descanso': '2 min'},
    {'nombre': 'Extensión de cuadriceps',
     'series': '4',
     'cargas': '0 %',
     'kilos': '50 Kg',
     'repeticiones': '10',
     'rpe': '8',
     'descanso': '1 min'},
    {'nombre': 'Femoral en maquina',
     'series': '4',
     'cargas': '0 %',
     'kilos': '20 Kg',
     'repeticiones': '10',
     'rpe': '7',
     'descanso': '1 min'},
    {'nombre': 'Abdominales crunch (Solo 15º) ',
     'series': '5',
     'cargas': '70 %',
     'kilos': '2 Kg',
     'repeticiones': '15',
     'rpe': '7',
     'descanso': '1 min'}]},
  {'wor

# 2. Show data

In [7]:
for block_name, sessions_list in micro_dict.items():
    print(block_name)
    print("-"*30)
    # print(sessions_list)
    for session in sessions_list:
        print(f"{session['date_workout']} - {session['workout_desc']}")
        display(pd.DataFrame(session["exercises"]))
        

Micro 0 Gonzalo septiembre2021
------------------------------
2021-09-06 - pierna 


,nombre,series,cargas,kilos,repeticiones,rpe,descanso
0,Sentadilla,4,0 %,70 Kg,10,8,2 min
1,Peso muerto,4,0 %,100 Kg,5,5,2 min
2,Extensión de cuadriceps,4,0 %,50 Kg,10,8,1 min
3,Femoral en maquina,4,0 %,20 Kg,10,7,1 min
4,Abdominales crunch (Solo 15º),5,70 %,2 Kg,15,7,1 min


2021-09-07 - torso


,nombre,series,cargas,kilos,repeticiones,rpe,descanso
0,Press de banca,4,60 %,60 Kg,10,7,2 min
1,Press plano mancuernas,4,70 %,25 Kg,10,7,2 min
2,Dominadas libres,4,70 %,80 Kg,8,6,2 min
3,Remo con barra,4,50 %,50 Kg,10,5,1 min
4,Press francés Barra olímpica,5,70 %,20 Kg,10,5,1 min


2021-09-09 - acc pierna pst


,nombre,series,cargas,kilos,repeticiones,rpe,descanso
0,Peso muerto rumano,4,0 %,50 Kg,12,7,1 min
1,Peso muerto rumano con mancuernas,4,0 %,20 Kg,10,8,1 min
2,Hip thrust,4,0 %,100 Kg,8,6,2 min
3,Sentadilla búlgara (2 Mancuernas de 10),4,0 %,10 Kg,8,7,1 min
4,Gemelo en multipower,5,0 %,60 Kg,12,6,1 min


2021-09-11 - acc torso


,nombre,series,cargas,kilos,repeticiones,rpe,descanso
0,Press inclinado,4,50 %,50 Kg,12,6,2 min
1,Press militar,4,70 %,30 Kg,12,8,2 min
2,Press militar mancuernas sentado con respaldo,4,70 %,10 Kg,12,6,1 min
3,Jalón al pecho,4,70 %,40 Kg,12,7,1 min
4,Curl de biceps barra olímpica,4,70 %,20 Kg,12,8,1 min
5,Curl de biceps mancuernas,4,70 %,10 Kg,10,7,1 min
6,Face pull,5,70 %,10 Kg,15,8,1 min
7,Rotación externa de hombro en polea,5,70 %,5 Kg,15,9,1 min


In [1]:
soup.find_all

NameError: name 'soup' is not defined